In [1]:
import os
import random
import shutil
import time
import warnings

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim

import torch.utils.data
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

import torch.distributed as dist

In [2]:
SEED=1
random.seed(SEED)
torch.manual_seed(SEED)
cudnn.deterministic = True

In [3]:
torch.cuda.device_count()

1

In [4]:
START_EPOCH = 0
ARCH = 'resnet18'
EPOCHS = 200
LR = 0.1
MOMENTUM = 0.9
WEIGHT_DECAY = 1e-4
PRINT_FREQ = 50
TRAIN_BATCH=256
VAL_BATCH=256
WORKERS=8
TRAINDIR="/workspace/storage/train"
VALDIR="/workspace/storage/val"

In [5]:
if not torch.cuda.is_available():
    print('GPU not detected.. did you pass through your GPU?')

In [6]:
URL = 'tcp://3.19.70.152:1234'
BACKEND = 'nccl'
WORLD_SIZE = 2
RANK = 1

dist.init_process_group(backend = BACKEND, init_method= URL,
                                 world_size= WORLD_SIZE, rank=RANK)

In [7]:
GPU = 0
torch.cuda.set_device(GPU)
cudnn.benchmark = True

In [8]:
def train(train_loader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, data_time, losses, top1, top5],
        prefix="Epoch: [{}]".format(epoch))

    # switch to train mode
    model.train()

    end = time.time()
    for i, (images, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if GPU is not None:
            images = images.cuda(GPU, non_blocking=True)
        if torch.cuda.is_available():
            target = target.cuda(GPU, non_blocking=True)

        # compute output
        output = model(images)
        loss = criterion(output, target)

        # measure accuracy and record loss
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        losses.update(loss.item(), images.size(0))
        top1.update(acc1[0], images.size(0))
        top5.update(acc5[0], images.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % PRINT_FREQ == 0:
            progress.display(i)

In [9]:
def validate(val_loader, model, criterion):
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, losses, top1, top5],
        prefix='Test: ')

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (images, target) in enumerate(val_loader):
            if GPU is not None:
                images = images.cuda(GPU, non_blocking=True)
            if torch.cuda.is_available():
                target = target.cuda(GPU, non_blocking=True)

            # compute output
            output = model(images)
            loss = criterion(output, target)

            # measure accuracy and record loss
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))
            top5.update(acc5[0], images.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % PRINT_FREQ == 0:
                progress.display(i)

        # TODO: this should also be done with the ProgressMeter
        print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))

    return top1.avg

In [10]:
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')

In [11]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

In [12]:
class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

In [13]:
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    #lr = LR * (0.1 ** (epoch // 30))
    lr = LR * (0.1 ** (epoch // 17))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [14]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [15]:
imagenet_mean_RGB = [0.47889522, 0.47227842, 0.43047404]
imagenet_std_RGB = [0.229, 0.224, 0.225]
cinic_mean_RGB = [0.47889522, 0.47227842, 0.43047404]
cinic_std_RGB = [0.24205776, 0.23828046, 0.25874835]
cifar_mean_RGB = [0.4914, 0.4822, 0.4465]
cifar_std_RGB = [0.2023, 0.1994, 0.2010]

In [16]:
normalize = transforms.Normalize(mean=imagenet_mean_RGB, std=imagenet_std_RGB)

In [17]:
IMG_SIZE = 224
NUM_CLASSES = 1000
model = models.__dict__[ARCH]()
inf = model.fc.in_features
model.fc = nn.Linear(inf, NUM_CLASSES)
model.cuda(GPU)
model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[GPU])

In [18]:
criterion = nn.CrossEntropyLoss().cuda(GPU)
optimizer = torch.optim.SGD(model.parameters(), LR,
                                momentum=MOMENTUM,
                                weight_decay=WEIGHT_DECAY)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(cinic_mean_RGB, cinic_std_RGB),
])

In [19]:
# train_dataset = datasets.ImageFolder(
#     TRAINDIR, transform=transform_train)

train_dataset = datasets.ImageFolder(
    TRAINDIR,
    transforms.Compose([
        transforms.Resize(256),
        transforms.RandomResizedCrop(224),
        transforms.CenterCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ])
)

In [20]:
# transform_val = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize(cinic_mean_RGB, cinic_std_RGB),
# ])

val_dataset = datasets.ImageFolder(
    VALDIR,
    transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize,
    ])
)

In [21]:
train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=TRAIN_BATCH, shuffle=False,
        num_workers=WORKERS, pin_memory=True, sampler=torch.utils.data.distributed.DistributedSampler(train_dataset))

In [22]:
val_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=VAL_BATCH, shuffle=False,
        num_workers=WORKERS, pin_memory=True, sampler=None) 

In [23]:
best_acc1 = 0

for epoch in range(START_EPOCH, EPOCHS):
    
    t1 = time.time()
    
    #Implement Learning Rate Adjustment
    adjust_learning_rate(optimizer, epoch)

    # train for one epoch
    train(train_loader, model, criterion, optimizer, epoch)

    # evaluate on validation set
    acc1 = validate(val_loader, model, criterion)

    # remember best acc@1 and save checkpoint
    is_best = acc1 > best_acc1
    best_acc1 = max(acc1, best_acc1)


    save_checkpoint({
        'epoch': epoch + 1,
        'arch': ARCH,
        'state_dict': model.state_dict(),
        'best_acc1': best_acc1,
        'optimizer' : optimizer.state_dict(),
    }, is_best)
    
    scheduler.step()
    print('lr: ' + str(scheduler.get_last_lr()))
    
    t2 = time.time()
    
    print("Epoch Time:",(t2-t1)/60,"minutes")

Epoch: [0][   0/2503]	Time 17.952 (17.952)	Data  3.847 ( 3.847)	Loss 7.0251e+00 (7.0251e+00)	Acc@1   0.00 (  0.00)	Acc@5   0.39 (  0.39)
Epoch: [0][  50/2503]	Time  0.786 ( 1.099)	Data  0.000 ( 0.080)	Loss 6.7423e+00 (6.9377e+00)	Acc@1   0.78 (  0.21)	Acc@5   3.52 (  1.02)
Epoch: [0][ 100/2503]	Time  0.814 ( 0.952)	Data  0.000 ( 0.046)	Loss 6.6297e+00 (6.8077e+00)	Acc@1   0.39 (  0.47)	Acc@5   3.12 (  1.83)
Epoch: [0][ 150/2503]	Time  1.699 ( 0.989)	Data  0.000 ( 0.035)	Loss 6.5039e+00 (6.7087e+00)	Acc@1   0.78 (  0.65)	Acc@5   3.52 (  2.48)
Epoch: [0][ 200/2503]	Time  1.589 ( 1.071)	Data  0.000 ( 0.031)	Loss 6.4395e+00 (6.6247e+00)	Acc@1   0.00 (  0.85)	Acc@5   3.12 (  3.12)
Epoch: [0][ 250/2503]	Time  1.330 ( 1.132)	Data  0.000 ( 0.028)	Loss 6.2302e+00 (6.5439e+00)	Acc@1   0.78 (  1.04)	Acc@5   5.86 (  3.73)


/opt/conda/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


Epoch: [0][ 300/2503]	Time  1.275 ( 1.166)	Data  0.000 ( 0.027)	Loss 6.1233e+00 (6.4709e+00)	Acc@1   2.34 (  1.21)	Acc@5   5.47 (  4.32)
Epoch: [0][ 350/2503]	Time  1.226 ( 1.196)	Data  0.000 ( 0.026)	Loss 6.1139e+00 (6.4073e+00)	Acc@1   2.34 (  1.40)	Acc@5   7.03 (  4.86)
Epoch: [0][ 400/2503]	Time  1.184 ( 1.218)	Data  0.000 ( 0.026)	Loss 5.7536e+00 (6.3457e+00)	Acc@1   4.30 (  1.57)	Acc@5  11.33 (  5.40)
Epoch: [0][ 450/2503]	Time  1.263 ( 1.235)	Data  0.000 ( 0.025)	Loss 5.8076e+00 (6.2892e+00)	Acc@1   1.56 (  1.77)	Acc@5   8.59 (  6.00)
Epoch: [0][ 500/2503]	Time  1.046 ( 1.248)	Data  0.000 ( 0.025)	Loss 5.7697e+00 (6.2324e+00)	Acc@1   2.34 (  1.95)	Acc@5  10.55 (  6.62)
Epoch: [0][ 550/2503]	Time  1.312 ( 1.259)	Data  0.000 ( 0.024)	Loss 5.6152e+00 (6.1797e+00)	Acc@1   5.86 (  2.19)	Acc@5  14.06 (  7.21)
Epoch: [0][ 600/2503]	Time  1.700 ( 1.268)	Data  0.000 ( 0.024)	Loss 5.5690e+00 (6.1310e+00)	Acc@1   3.52 (  2.39)	Acc@5  11.72 (  7.80)
Epoch: [0][ 650/2503]	Time  1.842 ( 1.277

Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/multiprocessing/queues.py", line 245, in _feed
    send_bytes(obj)
  File "/opt/conda/lib/python3.8/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/multiprocessing/queues.py", line 245, in _feed
    send_bytes(obj)
  File "/opt/conda/lib/python3.8/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/opt/conda/lib/python3.8/multiprocessing/connection.py", line 411, in _send_bytes
    self._send(header + buf)
  File "/opt/conda/lib/python3.8/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
  File "/opt/conda/lib/python3.8/multiprocessing/connection.py", line 411, in _send_bytes
    self._send(header + buf)
  File "/opt/conda/lib/python3.8/multiprocessing/connection.py", 

KeyboardInterrupt: 